In [1]:
from llama_index.llms.ollama import Ollama
MODEL="llama3.1"
PROMPT= "Be a good boy"
DATA_PATH="./db.txt"

In [2]:
llm = Ollama(model=MODEL)

In [3]:
def retrieve_data(query):
    with open(DATA_PATH, 'r') as file:
        file_content = file.read()
    return file_content
    
# Step 3: Define the RAG pipeline with retrieval and generation
def rag_pipeline(query):
    # Retrieve relevant documents
    # retrieved_docs = [retrieve_data(query)]
    data = ""
    data = retrieve_data(query)
    # Build the prompt with the retrieved documents
    prompt = PROMPT + data + query
    # print(prompt)
    # Generate the response with Ollama
    response = llm.complete(prompt)
    return response  # Extracting the generated answer text



In [4]:
# Test the RAG pipeline
# query = "Give me insight about the main KPIs?"
query = "Give me insights about the important KPIs"

answer = rag_pipeline(query)
print(answer)

Based on the provided information, here are some insights about the important KPIs:

**Time-Related KPIs**

1. **working_time**: This KPI measures the active machine operation time in seconds. It's essential to monitor this metric as it directly impacts productivity and efficiency.
2. **idle_time**: The idle time KPI tracks when machines are not operational, indicating potential inefficiencies or maintenance needs.
3. **offline_time**: Offline time is critical to understand, as it can lead to production delays and revenue losses.

**Energy and Cost-Related KPIs**

1. **consumption**: This KPI measures the total energy consumption in kWh, which is crucial for understanding power costs and environmental impact.
2. **power**: The power rating (kW) helps evaluate machine performance and potential energy efficiency improvements.
3. **cost**: Energy cost per kWh is a vital metric to monitor, as it directly affects business expenses.
4. **consumption_working**: Tracking consumption during act